In [1]:
from __future__ import absolute_import, print_function
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

c:\Users\Noel\anaconda3\envs\torchCUDA\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


In [2]:
weights = None  # ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)

# reshape last layer.
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 10)
model.load_state_dict(torch.load(r"C:\Users\Noel\Documents\THESIS\Feature Visualization\Weights\resnet18_torchvision\test52_epoch298.pth"))
# model.load_state_dict(torch.load(r"C:\Users\Noel\Documents\THESIS STUFF\PYTHON-THINGIES\Saved Model Parameters\resnet18_torchvision\test39_epoch348.pth"))
# Set model to evaluation mode and send to device
model.to(device).eval()

layers_of_interest = [name for name, _ in model.named_modules() if "conv" in name or "fc" in name]

print(layers_of_interest)


['conv1', 'layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2', 'fc']


This cell works, but is technically wrong and would not 'fly' if the notebook was to be converted to an actual python script.

In [3]:
# layer_activations = {}

# def hook_wrapper(name: str):
#     def hook_fn(module: nn.Module, input: torch.Tensor, output: torch.Tensor) -> None:
#         layer_activations[name] = output
#     return hook_fn


# for name, layer in model.named_modules():
#     if name in layers_of_interest:
#         layer.register_forward_hook(hook_wrapper(name))

To combat this, I made the hooks into objects of a hook class, holding both the output and the hook function, thus creating dictionary entries 
of a key/value pair of name/Hook_Layer object.

In [4]:
class Hook_Layer():
    def __init__(self, layer) -> None:
        self.hook = layer.register_forward_hook(self.hook_fn)
        self.output = None

    def hook_fn(self, layer, input, output):
        self.output = output
    
    def __call__(self):
        return self.output

layer_activations = {}
for name, layer in model.named_modules():
    if name in layers_of_interest:
        layer_activations[name] = Hook_Layer(layer)

print(layer_activations.values())

dict_values([<__main__.Hook_Layer object at 0x000001F0E769C550>, <__main__.Hook_Layer object at 0x000001F0F27EFEE0>, <__main__.Hook_Layer object at 0x000001F0F27ED030>, <__main__.Hook_Layer object at 0x000001F0F27EFB50>, <__main__.Hook_Layer object at 0x000001F0F27EFA60>, <__main__.Hook_Layer object at 0x000001F0F27EDC60>, <__main__.Hook_Layer object at 0x000001F0F27EDD50>, <__main__.Hook_Layer object at 0x000001F0F27EF2E0>, <__main__.Hook_Layer object at 0x000001F0F27EF3D0>, <__main__.Hook_Layer object at 0x000001F0F27EE8F0>, <__main__.Hook_Layer object at 0x000001F0F27EEA70>, <__main__.Hook_Layer object at 0x000001F0F27EEB90>, <__main__.Hook_Layer object at 0x000001F0F27EECB0>, <__main__.Hook_Layer object at 0x000001F0F27EEE30>, <__main__.Hook_Layer object at 0x000001F0F27EEF50>, <__main__.Hook_Layer object at 0x000001F0F27EF070>, <__main__.Hook_Layer object at 0x000001F0F27EF1F0>, <__main__.Hook_Layer object at 0x000001F0F27EF820>])


In [5]:
# Create a dataset class that extends ImageFolder while
# simultaneously returning a 3 way Tuple, instead of the
# original that contains 2 elements.
# For that reason we must define a new __getitem__ method.
class ImageFolderWithPaths(ImageFolder):
    """Dataset class extending ImageFolder dataset,
        returning Tuple.
        
        Returns:
                Tuple[img[torch.Tensor],
                      label[int],
                      path[str]]
        """
    def __getitem__(self, index: int):
        # Super the __getitem__ of base class
        img, label = super().__getitem__(index)
        # Extract the path of each image in the dataset
        path = self.imgs[index][0]
        # Return new tuple with 3 elements
        return (img, label, path)

    

In [6]:
batch_size = 64


transforms = T.Compose([T.Resize(224),
                        T.CenterCrop(224),
                        T.ToTensor(),
                        T.Normalize([0.5162, 0.4644, 0.3975],
                                    [0.2724, 0.2640, 0.2574])
                        ])

dataset = ImageFolderWithPaths(root=r"C:\Users\Noel\Documents\THESIS\Data\artbench-10-imagefolder-split\train",
                               transform=transforms)
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False, num_workers=0)
print("Dataloader Initialized. Note that workers tend to take some time to \
initialize but speed up performance when loading.")
# ================================================================
data = []

with torch.no_grad():
    for images, labels, paths in tqdm(dataloader, total=len(dataloader)):
        # Send stuff to GPU if available.
        images = images.to(device)
        labels = labels.to(device)
        # Make Forward Pass.
        outputs = model(images)
        _, preds = torch.max(outputs, dim=1)
        # path_list = []
        # for path in paths:
        #     path_list.append(path)

        for i, image in enumerate(images):
            private_dict = {}
            # Three entries regarding the image identification.
            # private_dict['path'] = path_list[i]
            private_dict['path'] = paths[i]
            private_dict['class_label'] = labels[i].item()
            private_dict['prediction'] = preds[i].item()
            # Iterate over all available layers.
            for key, hook_object in layer_activations.items():
                tensor_out = hook_object()  # .output  # modified from original script to accommodate objects 
                if key == 'fc':
                    # The array to store is a 32 by 10 array, each batch
                    output = torch.unbind(tensor_out, dim=0)
                else:
                    # The array will have a final shape of 32 by num_channels
                    # in specific layer
                    b, c, _, _ = tensor_out.shape
                    output = torch.unbind(tensor_out.view(b, c, -1).mean(2), dim=0)
                private_dict[key] = output[i].cpu().numpy()
            data.append(private_dict)

Dataloader Initialized. Note that workers tend to take some time to initialize but speed up performance when loading.


100%|██████████| 782/782 [13:50<00:00,  1.06s/it]


In [7]:
df = pd.DataFrame(data, copy=False)
# df.head()

In [8]:
df.to_parquet('test52_activations.parquet')

In [9]:
df = pd.read_parquet('test52_activations.parquet')
df.head()

,path,class_label,prediction,conv1,layer1.0.conv1,layer1.0.conv2,layer1.1.conv1,layer1.1.conv2,layer2.0.conv1,layer2.0.conv2,...,layer2.1.conv2,layer3.0.conv1,layer3.0.conv2,layer3.1.conv1,layer3.1.conv2,layer4.0.conv1,layer4.0.conv2,layer4.1.conv1,layer4.1.conv2,fc
0,C:\Users\Noel\Documents\THESIS\Data\artbench-1...,0,0,"[0.0018607399, 0.0016387592, -0.16810241, -0.0...","[-0.07074278, -0.04438116, -0.01781173, -0.010...","[-0.0065847305, -0.0027117555, -0.0026636445, ...","[-0.009212078, 0.014996161, -0.008652206, -0.0...","[0.0017869333, -0.0019380569, -0.00026760218, ...","[-0.018156525, -0.015329811, -0.0037448092, -0...","[0.0016469071, 0.0016380888, -0.005591418, -0....",...,"[-0.002437883, -0.0013578436, -0.00318265, -0....","[-0.002879563, -0.0028790524, -0.0015525137, -...","[-0.0030659016, -0.010099288, -0.005509351, -0...","[-0.0046330816, -0.005344051, -0.0032269838, -...","[-6.7617816e-07, 0.0011859337, -7.8081524e-05,...","[-0.007940755, -0.0033864488, -0.0076252087, -...","[0.021948624, 0.0017738822, -0.0013834587, -0....","[-0.02885245, -0.03397983, 0.03010794, 0.00912...","[0.03688384, 0.0011115115, -0.00563405, -0.007...","[5.0417814, -1.8061397, -0.16598052, 0.0332453..."
1,C:\Users\Noel\Documents\THESIS\Data\artbench-1...,0,0,"[0.004358571, 0.0012037483, -0.30539873, -0.02...","[-0.067148365, -0.045166656, -0.019599574, -0....","[-0.0074839094, -0.0033091798, 0.0010164687, 0...","[-0.017039208, 0.02174373, -0.0125999665, -0.0...","[0.0023305477, -0.0021498264, -0.00015958618, ...","[-0.020575285, -0.01957996, -0.00533754, -0.01...","[-0.00035945605, 0.0043217717, -0.004949149, -...",...,"[-0.0041252775, -0.0009423351, -0.0030982615, ...","[-0.0046263696, -0.0064888415, 0.00073228363, ...","[-0.0029584144, -0.008804231, 0.0011378769, -0...","[-0.0039636977, -0.0007523224, -0.0010057949, ...","[-0.0005691788, 0.000865072, -0.000712064, 0.0...","[-0.006963921, -0.004752762, -0.009064357, -0....","[0.023577694, -0.0026831522, -0.0058389185, -0...","[-0.044678517, -0.042343553, 0.017180633, 0.00...","[0.033136133, -0.006334798, -0.008237878, -0.0...","[5.2985783, -1.6083739, 0.7092223, -0.16076796..."
2,C:\Users\Noel\Documents\THESIS\Data\artbench-1...,0,3,"[-0.005771032, 0.0058223037, -0.06431603, -0.0...","[-0.061438084, -0.038136356, -0.01060826, -0.0...","[-0.003932758, -0.0022467943, 0.00084953057, 0...","[-0.009434916, 0.018614903, -0.014748029, -0.0...","[0.0023025924, -0.0020280792, -0.0009575669, -...","[-0.016995056, -0.02160984, -0.006406614, -0.0...","[0.0055686976, 0.0033957749, -0.0053579854, -0...",...,"[-0.0027628425, -0.001177982, -0.0012759076, -...","[-0.0014531022, -0.0059660044, 0.00023891883, ...","[-0.0010516944, -0.010817479, 0.0019851883, -0...","[-0.0015836364, -0.0047522313, 0.0019365733, -...","[0.000408507, -0.00051451894, 0.00024013115, 6...","[-0.00477473, -0.0029919366, -0.00843688, -0.0...","[-0.0008656288, -0.0015050092, -0.0031352136, ...","[0.013212544, -0.035520963, 0.012013794, -0.00...","[-0.003354337, -0.003095891, -0.006906514, 0.0...","[-0.10315622, -1.3674815, -0.050703093, 3.0977..."
3,C:\Users\Noel\Documents\THESIS\Data\artbench-1...,0,0,"[0.0014272248, -0.008437143, -0.039292026, 0.0...","[-0.057881206, -0.038057372, -0.014808282, -0....","[-0.0038383475, -0.0017724222, 0.0014776934, 0...","[-0.00673736, 0.018084116, -0.009799562, -0.01...","[0.0019012184, -0.0025287776, -0.00061434775, ...","[-0.020976124, -0.018771019, -0.008385152, -0....","[0.00067375164, 0.004215407, -0.005762818, -0....",...,"[-0.004282161, -0.0015523657, -0.0021147246, -...","[-0.0034086432, -1.1455723e-05, -5.131285e-05,...","[-0.0042324234, -0.0073553394, -0.005111066, 0...","[0.00088831276, -0.0035172245, -0.0003597472, ...","[0.00023111851, 0.00089437544, -0.00073870056,...","[-0.0049804817, -0.005219142, -0.0053514373, -...","[0.01887069, -0.0023917938, 0.0021667115, 0.00...","[-0.03578112, -0.025038539, 0.019966295, 0.012...","[0.03295044, -0.0056390525, 0.0018000014, -0.0...","[4.63150